In [1]:
import skimage
import matplotlib.pyplot as plt
import numpy as np
import pywt
import scipy
import os
import csv

from skimage import io, filters
from sklearn import linear_model, datasets
from PIL import Image

In [2]:
def get_noise_image(image):
    ret = np.zeros(image.shape, dtype='uint8')
    ret[:,:,0] = skimage.filters.rank.median(image[:,:,0], np.ones((5,5)))
    ret[:,:,1] = skimage.filters.rank.median(image[:,:,1], np.ones((5,5)))
    ret[:,:,2] = skimage.filters.rank.median(image[:,:,2], np.ones((5,5)))

    return image.astype('float64')/255.0 - ret.astype('float64')/255.0

In [3]:
def extract_features(image):
    subimages = []
    
    noise = get_noise_image(image)
    subimages.append(noise)

    coeffs = pywt.dwt2(noise, 'db2')

    subimages.append(coeffs[0])
    subimages.append(coeffs[1][0])
    subimages.append(coeffs[1][1])
    subimages.append(coeffs[1][2])

    line = []
    for im in subimages:
        for c in range(3):
            nobs, minmax, mean, variance, skewness, kurtosis = scipy.stats.describe(im[:,:,c].ravel())    
            line = line + [minmax[0],minmax[1],mean,variance,skewness,kurtosis]
    return line

In [4]:
logistic = linear_model.LogisticRegression()

#pca = decomposition.PCA()
#pipe = Pipeline(steps=[('pca', pca), ('logistic', logistic)])

X = []
y = []

# Load csv
with open('features.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        X.append(row[:-1])
        y.append(row[-1])

#pca.fit(X)

# Prediction
#n_components = [20, 40, 64]
#Cs = np.logspace(-4, 4, 3)

# Parameters of pipelines can be set using ‘__’ separated parameter names:
#estimator = GridSearchCV(pipe, dict(pca__n_components=n_components,logistic__C=Cs))
#estimator.fit(X_digits, y_digits)

logreg = linear_model.LogisticRegression(C=1e5)

# we create an instance of Neighbours Classifier and fit the data.

logreg.fit(X, y)


LogisticRegression(C=100000.0, class_weight=None, dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
          solver='liblinear', tol=0.0001, verbose=0, warm_start=False)

In [6]:
# Test data
labels = ['HTC-1-M7','iPhone-4s', 'iPhone-6', 'LG-Nexus-5x','Motorola-Droid-Maxx','Motorola-Nexus-6','Motorola-X','Samsung-Galaxy-Note3','Samsung-Galaxy-S4','Sony-NEX-7']

X = []
#i = 0
for image in os.listdir('test/'):
#    i+=1
#    if i > 11:
#        break
    im = Image.open('test/'+image)
    image = np.array(im).astype('uint8')
    
    X.append(extract_features(image))

array(['9.000000000000000000e+00', '7.000000000000000000e+00',
       '9.000000000000000000e+00', ..., '1.000000000000000000e+00',
       '7.000000000000000000e+00', '0.000000000000000000e+00'],
      dtype='<U24')

In [64]:
# Save coefficients
print(logreg.coef_)
np.savetxt("coefficients.csv", logreg.coef_, delimiter=",")

[[ 2.12073994e+00  2.30644808e+00  5.63162823e-02 -1.22527553e+00
  -1.26315107e-01  4.83971504e-02  5.70396345e+00  2.57268464e+00
  -1.11421863e+00 -1.34283321e+00 -8.84021516e-01  7.13507117e-02
  -7.70276212e+00 -4.40869988e+00 -1.38557414e+00 -1.29007603e+00
   1.81345792e+00 -1.60071028e-01 -1.96915592e+00 -1.48193651e+00
  -4.82766373e-01 -3.63334683e+00  4.21596158e-01 -3.65730489e-02
   6.40977031e+00 -4.83693317e-01 -1.47736145e+00 -3.75962643e+00
  -6.14290671e-03 -5.23945179e-02 -5.92116503e+00 -8.77912071e-01
  -2.96250832e+00 -3.90259432e+00 -7.91406183e-01  1.13482061e-01
   2.37434565e+00  4.99014403e-01 -2.70525222e+00 -1.29464109e+00
  -1.78189411e+00  7.40794774e-02  3.66880654e+00  5.25004473e+00
  -2.69264820e+00 -1.28860384e+00  2.71150044e+00 -9.73000359e-02
  -2.20189737e+00 -4.63716075e+00 -2.65846246e+00 -1.28920237e+00
  -1.50825580e+00  4.28799524e-02 -5.10870370e+00  1.92301264e+00
   1.01581150e+00 -8.19554984e-02 -3.85817190e-01  5.11491617e-03
  -9.04662

In [27]:
# Validation
X = []
y = []

# Load csv
with open('features.csv', newline='') as csvfile:
    spamreader = csv.reader(csvfile, delimiter=',')
    for row in spamreader:
        X.append(row[:-1])
        y.append(row[-1])

In [155]:
vX = np.matrix(X).astype('float64')
vy = np.matrix(y).astype('float64')
print(vX.shape)
print(vy.transpose().shape)

v_matrix = np.append(vX,vy.transpose(),axis=1).astype('float64')

np.random.shuffle(v_matrix)
v_matrix = v_matrix[:int(len(vX)/5)]

predicted = np.matrix(logreg.predict(v_matrix[:,:-1]).astype('float'))
result = v_matrix[:,-1].astype('float')
compare = np.append(predicted.transpose(), result, axis=1)

corrects = 0.0
for c in compare:
    if(c[0,0] == c[0,1]):
        corrects += 1.0
        
print("Accuracy:", corrects/v_matrix.shape[0])

(2750, 90)
(2750, 1)
Accuracy: 0.5236363636363637
